In [11]:
import pandas as pd
import numpy as np
train_op=pd.read_csv(r"D:\DA_competition\DC\data\operation_train.csv")
train_tag=pd.read_csv(r"D:\DA_competition\DC\data\tag_train.csv")
train_tr=pd.read_csv(r"D:\DA_competition\DC\data\transaction_train.csv")

#表连接
train_op=pd.merge(train_op,train_tag,on="UID",how="left")
train_op=train_op.drop_duplicates()
train_tr=pd.merge(train_tr,train_tag,on="UID",how="left")
train_tr=train_tr.drop_duplicates()

test_op=pd.read_csv(r"D:\DA_competition\DC\data\operation_test.csv")
test_tr=pd.read_csv(r"D:\DA_competition\DC\data\transaction_test.csv")
test_op=test_op.drop_duplicates()
test_tr=test_tr.drop_duplicates()


D:\developsoft\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [19]:
train_fts=pd.DataFrame()
test_fts=pd.DataFrame()
#31198

31198

In [9]:
#UID为主键
train_fts_discrete=pd.DataFrame()
test_fts_discrete=pd.DataFrame()
op_discrete_fields=["mode","success","os","version","device1","device2","geo_code"]
tr_discrete_fields=["channel","amt_src1","amt_src2","merchant","code1","code2","trans_type1","trans_type2","device1","device2","geo_code","market_code","market_type"]
for x in [[train_fts_discrete,train_op,train_tr],[test_fts_discrete,test_op,test_tr]]:
    fts_set=x[0]
    data_op=x[1]
    data_tr=x[2]
    fields=[op_discrete_fields,tr_discrete_fields]
    datas=[data_op,data_tr]
    data_name=["op","tr"]
    for i in [0,1]:
        data=datas[i].fillna(0)
        field=fields[i]
        for f in field:
            d_group=data.groupby("UID")[f]
            fts_set[f+"_mode_"+data_name[i]]=d_group.apply(lambda x:x.value_counts().index[0])
            fts_set[f+"_kinds_"+data_name[i]]=d_group.unique().apply(lambda x:len(x))

In [10]:
train_fts_discrete

,mode_mode_op,mode_kinds_op,success_mode_op,success_kinds_op,os_mode_op,os_kinds_op,version_mode_op,version_kinds_op,device1_mode_op,device1_kinds_op,...,device1_mode_tr,device1_kinds_tr,device2_mode_tr,device2_kinds_tr,geo_code_mode_tr,geo_code_kinds_tr,market_code_mode_tr,market_code_kinds_tr,market_type_mode_tr,market_type_kinds_tr
UID,,,,,,,,,,,,,,,,,,,,,
10000,d25caee90b27fa9b,3,1.0,1,103,1,7.0.7,2,aca4977fbe8741e1,1,...,aca4977fbe8741e1,1.0,IPHONE 5,1.0,0,1.0,0,1.0,0.0,1.0
10001,c8741ce15ceac2a4,6,1.0,2,102,3,6.6.2,2,09baf2f39bc3dc86,3,...,09baf2f39bc3dc86,2.0,HUAWEI GRA-CL00,2.0,wssp,4.0,0,4.0,0.0,2.0
10002,d25caee90b27fa9b,7,1.0,1,102,2,7.0.9,2,09baf2f39bc3dc86,2,...,09baf2f39bc3dc86,2.0,KIW-AL10,2.0,wmgu,1.0,0,1.0,0.0,1.0
10003,c8741ce15ceac2a4,2,1.0,1,102,1,7.0.9,1,d2cf44cec09806cc,1,...,09baf2f39bc3dc86,2.0,HWI-AL00,2.0,0,2.0,0,1.0,0.0,1.0
10004,c8741ce15ceac2a4,9,1.0,2,102,1,7.0.5,2,09baf2f39bc3dc86,1,...,09baf2f39bc3dc86,2.0,MHA-AL00,2.0,wkrn,3.0,0,1.0,0.0,1.0
10006,79f8b86938180c3c,3,0.0,2,102,2,0,2,630a1adff2a87007,2,...,630a1adff2a87007,2.0,MI MAX 2,2.0,wws5,2.0,0,1.0,0.0,1.0
10007,c8741ce15ceac2a4,3,1.0,1,102,1,7.0.9,1,09baf2f39bc3dc86,1,...,09baf2f39bc3dc86,1.0,FRD-AL10,1.0,ws4m,1.0,0,1.0,0.0,1.0
10008,c8741ce15ceac2a4,8,1.0,2,102,2,7.0.9,3,d2cf44cec09806cc,2,...,d2cf44cec09806cc,1.0,VIVO Y37A,1.0,wmpd,2.0,0,3.0,0.0,2.0
10009,c8741ce15ceac2a4,2,1.0,1,102,1,7.0.5,2,09baf2f39bc3dc86,1,...,09baf2f39bc3dc86,1.0,BKL-AL00,1.0,0,2.0,0,2.0,0.0,2.0


In [ ]:
train_fts_sequent1=pd.DataFrame()
test_fts_sequent1